In [13]:
import pandas as pd
import requests
# get surburb list
post = []
postcode = pd.read_excel("../data/suburb.xltx")
surburb = postcode['Unnamed: 1']
surburb = surburb.dropna()
surburb = surburb.drop_duplicates().reset_index().drop('index',axis = 1).rename({'Unnamed: 1':'surburb'}, axis = 1)
surburb

,surburb
0,Albert Park-Middle Park-West St Kilda
1,Armadale
2,Carlton North
3,Carlton-Parkville
4,CBD-St Kilda Rd
...,...
142,Traralgon
143,Wanagaratta
144,Warragul
145,Warrnambool


In [10]:
missing = pd.read_csv('../data/curated/missing_houses.csv')
missing = missing.drop(['Unnamed: 0', 'count'], axis = 1)
suburb_list = []
for index, value in missing.iterrows():
    suburb = d[value['postcode']]
    suburb_list.append(suburb)

missing['suburb'] = suburb_list
missing

,postcode,suburb
0,3747,beechworth
1,3744,wandiligong
2,3644,cobram
3,3792,the patch
4,3779,marysville
...,...,...
195,3505,merbein south
196,3616,tatura
197,3842,churchill
198,3916,point leo


In [7]:
# get postcode and surburb corresponding list
suburb_postcode_df = pd.read_csv('../data/cleaned_complete.csv')[['postcode','surburb']].drop_duplicates()
d = dict()
for index, values in suburb_postcode_df.iterrows():
    d[values['postcode']] = values['surburb']

In [6]:
# get suburb id for each suburb
df2 = pd.read_csv("../data/suburb_id.csv")
df2['suburb'] = df2['suburb'].apply(lambda x:x.lower())
df2 

,suburb,suburb_id
0,albert park,797
1,middle park,26507
2,armadale,1587
3,carlton north,8247
4,carlton,8217
...,...,...
205,torquay,40507
206,traralgon,40687
207,warragul,42517
208,warrnambool,42697


In [16]:
import config

from twython import Twython, TwythonError

# create a Twython object by passing the necessary secret passwords
twitter = Twython(config.api_key)

AttributeError: module 'config' has no attribute 'api_key'

In [ ]:

result_list = []
for item in surburb['surburb']:
    suburbs = item.split("-")
    for suburb in suburb_list:
        suburb_replaced = suburb.replace(" ", "%20")
        print("request suburb id for: ", suburb_replaced)
        url = f"https://api.domain.com.au/v1/addressLocators?searchLevel=Suburb&suburb={suburb_replaced}&state=VIC"
        headers = {"accept": "application/json", "X-Api-Key": "key_a68db2bad8d7006e8aa1a5326d638dd8"}
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print("request failed", response.reason)
            continue
        data = response.json()
        suburb_id_list = [suburb_info["ids"][0]["id"] for suburb_info in data]
        for suburb_id in suburb_id_list:
            result_list.append([suburb, suburb_id])


In [61]:
pd.DataFrame(result_list, columns= ['surburb', 'suburb_id']).drop_duplicates()

,surburb,suburb_id
0,beechworth,3587
1,wandiligong,42077
2,cobram,9647
3,the patch,39857
4,marysville,25517
...,...,...
199,merbein south,26167
200,tatura,39327
201,churchill,9197
202,point leo,32957


In [62]:
pd.DataFrame(result_list, columns= ['surburb', 'suburb_id']).drop_duplicates().to_csv("../data/suburb_id_2.csv", index=False)

In [65]:
import requests
df2 = pd.read_csv("../data/suburb_id_2.csv")
headers = {"accept": "application/json", "X-Api-Key": "key_3672a55119b75bf70117593dbf4698ba"}
agent_list = []
for index, item in df2[69:].iterrows():
    suburb_id = item["suburb_id"]
    print(suburb_id)
    url = f"https://api.domain.com.au/v1/agencies?q=suburbId%3A{suburb_id}&pageNumber=1&pageSize=10000"
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print("request failed", response.reason)
        continue
    data = response.json()
    if len(data) == 0:
        break
    if isinstance(data, dict):
        agent_list.extend([data])
    else:
        agent_list.extend(data)
df = pd.DataFrame(agent_list)
df.to_csv('../data/agent_list_new.csv')

18597
17437
40807
26107
4557
44437
25037
18247
9507
35177
24657
29827
7797
31437
25497
30537
request failed Not Found
27687
45527
33647
19037
161877
request failed Not Found
9967
request failed Not Found
44257
10827
3457
20337
9617
31397
40857
request failed Not Found
42957
28297
33607
2917
22187
5917
14457
16487
30047
24347
31687
6657
22387
22367
45747
39917
5677
15257
9087
39797
21947
14677
request failed Not Found
4757
6537
request failed Not Found
45657
977
11367
24937
40637
7937
3597
request failed Not Found
30667
18327
10767
18617
33207
4777
20347
38717
27267
4907
31287
30677
22997
44087
28237
38767
41407
32887
41467
35917
23027
36137
44047
request failed Not Found
28157
51697
request failed Not Found
19387
39517
15287
request failed Not Found
2727
737
44567
request failed Not Found
32967
6477
28017
29707
26457
147107
request failed Not Found
10077
22457
16147
41637
32517
39687
29987
15737
24027
31847
12987
27947
7037
39407
3417
30237
34927
15477
9777
42317
request failed Not Fou

In [66]:
import requests
df2 = pd.read_csv("../data/suburb_id_2.csv")
headers = {"accept": "application/json", "X-Api-Key": "key_3672a55119b75bf70117593dbf4698ba"}
agent_list = []
for index, item in df2[:69].iterrows():
    suburb_id = item["suburb_id"]
    print(suburb_id)
    url = f"https://api.domain.com.au/v1/agencies?q=suburbId%3A{suburb_id}&pageNumber=1&pageSize=10000"
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print("request failed", response.reason)
        continue
    data = response.json()
    if len(data) == 0:
        break
    if isinstance(data, dict):
        agent_list.extend([data])
    else:
        agent_list.extend(data)
df = pd.DataFrame(agent_list)
df.to_csv('../data/agent_list_new2.csv')

3587
42077
9647
39857
25517
2247
29597
13607
4927
request failed Not Found
29267
request failed Not Found
32667
request failed Not Found
18017
19497
10057
19577
21587
45707
23447
4247
request failed Not Found
45037
3537
40977
35167
45007
29467
41257
request failed Not Found
30307
5297
request failed Not Found
16507
32077
1627
33827
7857
38477
20117
14357
request failed Not Found
13827
17457
24137
8057
161957
request failed Not Found
44237
44887
request failed Not Found
23837
8087
request failed Not Found
18577
request failed Not Found
30407
44737
4157
request failed Not Found
31517
22237
request failed Not Found
24367
45677
40407
5117
41887
request failed Not Found
8567
2057
43077
16747
42147
39377
8807
12277
9027
34667
18047
37877
35697


In [71]:
df = pd.read_csv('../data/agent_list_new2.csv')
df2 = pd.read_csv('../data/agent_list_new.csv')
pd.concat([df,df2]).reset_index().drop(['index',	'Unnamed: 0'], axis = 1)

,accountType,address1,address2,description,domainUrl,email,fax,hasRecentlySold,homePageSearchOptions,id,...,name,numberForRent,numberForSale,querySuburb,rentalEmail,rentalTelephone,showTabSoldLastYear,state,suburb,telephone
0,37,1/75 Ford Street,NaN,"<p mce_serialized=""4"">Devlin Beechworth real e...",devlinestateagents-18179,rod@devlins.com.au,NaN,True,0.000000e+00,18179,...,Devlin Estate Agents,3,4,Beechworth,rentals@devlins.com.au,03 5728 1999,True,VIC,Beechworth,03 5728 1999
1,5,29B Camp St,NaN,"<p class=""MsoNormal""> </p><p class=""MsoNormal""...",indigorealestate-25188,jamie@indigorealestate.com.au,NaN,True,NaN,25188,...,Indigo Real Estate,2,8,Beechworth,jamie@indigorealestate.com.au,03 5728 3295,True,VIC,Beechworth,03 5728 3295
2,37,76 Ford Street,NaN,After many years of gaining valuable experienc...,ravidarealestate-31475,sell@ravidarealestate.com.au,NaN,True,NaN,31475,...,Ravida Real Estate,1,17,Beechworth,sell@ravidarealestate.com.au,0438001225,True,VIC,Beechworth,0438001225
3,5,6/520 Swift Street,NaN,"<p><img mce_src=""http://images.domain.com.au/i...",brianunthankrealestate-11866,property@bur.com.au,02 60215016,True,1.111000e+11,11866,...,Brian Unthank Real Estate,3,21,NaN,property@bur.com.au,02 60413777,True,NSW,Albury,02 60413777
4,37,297 Schubach St,NaN,"<p mce_serialized=""106"">Elders is pleased to b...",eldersalburyruralbusinessalbury-1194,re_albury@elders.com.au,02 60412908,True,1.111000e+13,1194,...,Elders Albury Rural & Business Albury,0,12,NaN,DG_Real_Estate_Albury@elders.com.au,02 60213355,True,NSW,Albury,02 60213355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1175,1,55 Evans Street,NaN,"<p mce_serialized=""4""></p><p class=""MsoNormal""...",ypasunbury-18193,sunbury@ypa.com.au,NaN,True,0.000000e+00,18193,...,YPA SUNBURY,8,17,NaN,sunburyrentals@ypa.com.au,03 97444428,True,VIC,Sunbury,03 97444428
1176,37,91 High Street,NaN,"<p>Local property market experts, the Jellis C...",jelliscraigwoodend-22405,tommay@jelliscraig.com.au,03 54272811,True,NaN,22405,...,Jellis Craig Woodend,6,26,NaN,tommay@jelliscraig.com.au,03 54272800,True,VIC,Woodend,03 54272800
1177,5,83 High St,NaN,"<img mce_serialized=""4"" mce_src=""http://img.do...",keatingsrealestate-3401,email@keatings.com.au,03 5427 1611,True,1.100001e+13,3401,...,Keatings Real Estate,0,14,NaN,email@keatings.com.au,03 5427 2999,True,VIC,Woodend,03 5427 2999
1178,37,"Shop 3A, 81 High Street",NaN,<p>The “In Town” of the Macedon Ranges located...,rainehornewoodend-5272,info@woodend.rh.com.au,03 54273022,True,1.111000e+13,5272,...,Raine & Horne Woodend,0,6,NaN,info@woodend.rh.com.au,03 5427 3200,True,VIC,Woodend,03 5427 3200


In [70]:
def to_int(text):
    if len(text.split(' ')) == 1:
        return int(text)
    else:
        return 0

In [72]:
# df1 = pd.read_csv('../data/agent_list.csv',on_bad_lines='skip')
# df1 = df1[~(df1['numberForRent'].isnull())]
# df1['numberForRent'] = df1['numberForRent'].apply(to_int)
# df2 = pd.read_csv('../data/agent_list2.csv',on_bad_lines='skip')
# agent_info = pd.concat([df1, df2]).reset_index().drop(['index',	'Unnamed: 0'], axis = 1)
df = pd.read_csv('../data/agent_list_new2.csv')
df2 = pd.read_csv('../data/agent_list_new.csv')
agent_info = pd.concat([df,df2]).reset_index().drop(['index',	'Unnamed: 0'], axis = 1)
agent_info = agent_info[agent_info['numberForRent'] > 0]
agent_info_id = agent_info['id']
agent_info_id = agent_info_id.drop_duplicates()
agent_info_id.to_csv('../data/agent_info_id_new.csv')

In [73]:
import pandas as pd
agent_info_id = pd.read_csv('../data/agent_info_id_new.csv')
agent_info_id = agent_info_id.drop('Unnamed: 0', axis = 1)

In [74]:
agent_info_id

,id
0,18179
1,25188
2,31475
3,11866
4,21221
...,...
322,26155
323,28921
324,6924
325,6114


In [75]:
import requests
house_list = []
headers = {'accept': 'application/json','X-Api-key':'key_2064b0fe29a00f6a734ff057907cb717'}
for i in agent_info_id.id:
    print(i)
    url = f'https://api.domain.com.au/v1/agencies/{i}/listings?listingStatusFilter=liveAndArchived&pageNumber=1&pageSize=100000'
    response = requests.get(url,headers = headers)
    data = response.json()
    if len(data) == 0:
        continue
    if isinstance(data, dict):
        house_list.extend([data])
    else:
        house_list.extend(data)

18179
25188
31475
11866
21221
23637
23123
13814
22622
11851
11839
8626
27100
28091
24072
22614
23365
22827
26506
6643
8645
8644
6798
13691
13964
5808
6502
22037
14042
27443
8450
8448
8642
16716
27371
8176
18426
7279
29940
1195
27684
18609
31171
31813
25996
7658
19275
15849
15886
29574
32626
28544
28360
11641
4805
13719
17137
26551
3309
17380
3905
15409
21971
4698
18680
14969
30063
12997
4081
6472
12758
25162
20881
12871
17493
26485
15133
13136
28982
13462
16004
7555
26483
3400
24193
28273
8007
6724
21927
20341
22809
8231
3227
12967
8001
35122
15017
12946
29220
23851
18940
26960
4110
3370
11748
20643
33571
23610
12377
26089
8071
22369
8652
28817
20139
4668
3496
14588
8605
33202
21617
11931
15243
29607
12205
16084
17921
20375
22405
12617
13677
15213
13683
13411
7636
23154
11209
8650
11862
3399
22686
7742
4740
18193
5636
34189
18424
13170
18713
17285
5584
15473
27085
15490
26526
23608
5904
7609
28272
20786
14811
30340
25904
33563
28586
12914
18340
26079
13463
17870
7316
17479
27082
16085


In [77]:
df = pd.DataFrame(house_list)
df.to_csv('../data/raw/house_list14.csv')

,objective,propertyTypes,status,saleMode,channel,addressParts,advertiserIdentifiers,bathrooms,bedrooms,carspaces,...,virtualTourUrl,features,landAreaSqm,dateAvailable,inspectionDetails,saleDetails,buildingAreaSqm,statementOfInformation,energyEfficiencyRating,highlights
0,rent,[house],archived,archived,residential,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,2.0,1.0,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,rent,[house],archived,archived,residential,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,3.0,0.0,...,,[Gas],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,rent,[house],archived,archived,residential,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",2.0,4.0,6.0,...,,"[Close to Schools, Ensuite, Dining Room, Dishw...",2000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,rent,[house],archived,archived,residential,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",2.0,3.0,2.0,...,,[Close to Schools],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,rent,[house],archived,archived,residential,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",2.0,3.0,2.0,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64364,sale,[house],sold,sold,residential,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,3.0,4.0,...,,"[Air Conditioning, Built-In Wardrobes, Close t...",NaN,NaN,"{'inspections': [], 'pastInspections': [], 'is...","{'saleMethod': 'privateTreaty', 'soldDetails':...",NaN,NaN,0.0,NaN
64365,sale,[house],sold,sold,residential,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,4.0,2.0,...,NaN,"[Air Conditioning, Built-In Wardrobes, Close t...",1591.0,NaN,"{'inspections': [], 'pastInspections': [], 'is...","{'saleMethod': 'privateTreaty', 'soldDetails':...",NaN,NaN,0.0,NaN
64366,sale,[vacantLand],archived,archived,residential,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",0.0,0.0,4.0,...,,NaN,NaN,NaN,NaN,"{'saleMethod': 'privateTreaty', 'tenderDetails...",NaN,NaN,NaN,NaN
64367,sale,[house],sold,sold,residential,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,3.0,2.0,...,NaN,"[Air Conditioning, Built-In Wardrobes, Close t...",NaN,NaN,"{'inspections': [], 'pastInspections': [], 'is...","{'saleMethod': 'privateTreaty', 'soldDetails':...",NaN,NaN,0.0,NaN


In [78]:
df = pd.DataFrame(house_list)
df.to_csv('../data/raw/house_list_new.csv')

In [ ]:
import pandas as pd
house_data = pd.read_csv('../data/raw/house_list.csv')
house_data2 = pd.read_csv('../data/raw/house_list2.csv')
house_data3= pd.read_csv('../data/raw/house_list3.csv')
house_data4= pd.read_csv('../data/raw/house_list4.csv')
house_data5= pd.read_csv('../data/raw/house_list5.csv')
house_data6= pd.read_csv('../data/raw/house_list6.csv')
house_data7= pd.read_csv('../data/raw/house_list7.csv')
house_data8= pd.read_csv('../data/raw/house_list8.csv')
house_data9= pd.read_csv('../data/raw/house_list9.csv')
house = pd.concat([house_data, house_data2, house_data3, house_data4, house_data5, house_data6, house_data7, house_data8, house_data9])

In [ ]:
house_data10 = pd.read_csv('../data/raw/house_list10.csv')
house_data11 = pd.read_csv('../data/raw/house_list11.csv')
house_data12 = pd.read_csv('../data/raw/house_list12.csv')
house_data13 = pd.read_csv('../data/raw/house_list13.csv')
house_data13 = house_data13[:5730]
house_data14= pd.read_csv('../data/raw/house_list14.csv')
house_2 = pd.concat([house_data10,house_data11, house_data12,house_data13,house_data14])

In [80]:
house = pd.read_csv('../data/raw/house_list_new.csv')
house

,Unnamed: 0,objective,propertyTypes,status,saleMode,channel,addressParts,advertiserIdentifiers,bathrooms,bedrooms,...,virtualTourUrl,features,landAreaSqm,dateAvailable,inspectionDetails,saleDetails,buildingAreaSqm,statementOfInformation,energyEfficiencyRating,highlights
0,0,rent,['house'],archived,archived,residential,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,rent,['house'],archived,archived,residential,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,3.0,...,NaN,['Gas'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,rent,['house'],archived,archived,residential,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",2.0,4.0,...,NaN,"['Close to Schools', 'Ensuite', 'Dining Room',...",2000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,rent,['house'],archived,archived,residential,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",2.0,3.0,...,NaN,['Close to Schools'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,rent,['house'],archived,archived,residential,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",2.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64366,64364,sale,['house'],sold,sold,residential,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,3.0,...,NaN,"['Air Conditioning', 'Built-In Wardrobes', 'Cl...",NaN,NaN,"{'inspections': [], 'pastInspections': [], 'is...","{'saleMethod': 'privateTreaty', 'soldDetails':...",NaN,NaN,0.0,NaN
64367,64365,sale,['house'],sold,sold,residential,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,4.0,...,NaN,"['Air Conditioning', 'Built-In Wardrobes', 'Cl...",1591.0,NaN,"{'inspections': [], 'pastInspections': [], 'is...","{'saleMethod': 'privateTreaty', 'soldDetails':...",NaN,NaN,0.0,NaN
64368,64366,sale,['vacantLand'],archived,archived,residential,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,"{'saleMethod': 'privateTreaty', 'tenderDetails...",NaN,NaN,NaN,NaN
64369,64367,sale,['house'],sold,sold,residential,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,3.0,...,NaN,"['Air Conditioning', 'Built-In Wardrobes', 'Cl...",NaN,NaN,"{'inspections': [], 'pastInspections': [], 'is...","{'saleMethod': 'privateTreaty', 'soldDetails':...",NaN,NaN,0.0,NaN


In [81]:
house_df = house[['propertyTypes','objective','addressParts','advertiserIdentifiers','bathrooms','bedrooms', 'carspaces','dateUpdated', 'dateListed','geoLocation','priceDetails','landAreaSqm','dateAvailable']]

In [ ]:
house_df_2 = house_2[['propertyTypes','objective','addressParts','advertiserIdentifiers','bathrooms','bedrooms', 'carspaces','dateUpdated', 'dateListed','geoLocation','priceDetails','landAreaSqm','dateAvailable']]
house_df_2

In [82]:
house_df = house_df[house_df['objective'] == 'rent']

In [ ]:
house_df_2 = house_df_2[house_df_2['objective'] == 'rent']
house_df_2

In [83]:
house_df = house_df.reset_index().drop('index', axis = 1)
house_df

,propertyTypes,objective,addressParts,advertiserIdentifiers,bathrooms,bedrooms,carspaces,dateUpdated,dateListed,geoLocation,priceDetails,landAreaSqm,dateAvailable
0,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,2.0,1.0,2012-05-30T14:45:58.36Z,2011-08-25T04:31:45Z,"{'latitude': -36.369734, 'longitude': 146.6766...","{'price': 275.0, 'canDisplayPrice': True, 'dis...",NaN,NaN
1,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,3.0,0.0,2012-05-02T01:12:41.443Z,2011-08-25T04:31:50Z,"{'latitude': -36.3718124, 'longitude': 146.645...","{'price': 260.0, 'canDisplayPrice': True, 'dis...",NaN,NaN
2,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",2.0,4.0,6.0,2011-10-11T12:34:45.29Z,2011-08-25T04:31:53Z,"{'latitude': -36.349709, 'longitude': 146.694623}","{'price': 500.0, 'canDisplayPrice': True, 'dis...",2000.00,NaN
3,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",2.0,3.0,2.0,2011-10-14T08:11:42.25Z,2011-08-25T04:31:55Z,"{'latitude': -36.353417, 'longitude': 146.696584}","{'price': 340.0, 'canDisplayPrice': True, 'dis...",NaN,NaN
4,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",2.0,3.0,2.0,2011-10-14T08:11:42.37Z,2011-08-30T19:31:22Z,"{'latitude': -36.353344, 'longitude': 146.695924}","{'price': 320.0, 'canDisplayPrice': True, 'dis...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
59058,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,4.0,4.0,2022-09-05T23:17:47.523Z,2022-08-05T03:33:29Z,"{'latitude': -37.2844387, 'longitude': 144.728...","{'canDisplayPrice': False, 'displayPrice': 'UN...",NaN,2022-08-11
59059,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,4.0,2.0,2022-09-27T23:34:22.997Z,2022-08-21T23:50:38Z,"{'latitude': -37.2844387, 'longitude': 144.728...","{'canDisplayPrice': False, 'displayPrice': 'UN...",567.91,2022-08-22
59060,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",1.0,2.0,0.0,2022-09-27T04:46:42.247Z,2022-09-25T23:29:15Z,"{'latitude': -37.3460627, 'longitude': 144.741...","{'canDisplayPrice': False, 'displayPrice': '$4...",NaN,2022-10-28
59061,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,4.0,4.0,2022-10-03T23:30:24.74Z,2022-09-26T03:48:38Z,"{'latitude': -37.3626928, 'longitude': 144.747...","{'canDisplayPrice': False, 'displayPrice': 'UN...",813.00,2022-10-03


In [421]:
house_df_2 = house_df_2.reset_index().drop('index', axis = 1)
house_df_2

,propertyTypes,objective,addressParts,advertiserIdentifiers,bathrooms,bedrooms,carspaces,dateUpdated,dateListed,geoLocation,priceDetails,landAreaSqm,dateAvailable
0,['apartmentUnitFlat'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,2.0,0.0,2005-07-20T10:01:03.573Z,2005-07-11T10:41:17Z,"{'latitude': -37.83775, 'longitude': 144.941043}","{'price': 600.0, 'canDisplayPrice': True, 'dis...",NaN,NaN
1,['apartmentUnitFlat'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",1.0,3.0,1.0,2005-08-18T06:05:13.013Z,2005-07-13T09:41:48Z,"{'latitude': -37.834815, 'longitude': 144.960958}","{'price': 400.0, 'canDisplayPrice': True, 'dis...",NaN,NaN
2,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,3.0,1.0,2005-07-19T05:28:18Z,2005-07-14T14:01:41Z,"{'latitude': -37.842189, 'longitude': 144.95752}","{'price': 550.0, 'canDisplayPrice': True, 'dis...",NaN,NaN
3,['apartmentUnitFlat'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",1.0,2.0,2.0,2005-07-22T11:35:45.783Z,2005-07-14T14:01:53Z,"{'latitude': -37.851494, 'longitude': 144.956552}","{'price': 330.0, 'canDisplayPrice': True, 'dis...",NaN,NaN
4,['apartmentUnitFlat'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,3.0,2.0,2007-08-28T00:49:03Z,2005-07-15T05:19:40Z,"{'latitude': -37.839805, 'longitude': 144.927453}","{'price': 875.0, 'canDisplayPrice': True, 'dis...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
152899,['house'],rent,"{'stateAbbreviation': 'nsw', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,2.0,1.0,2018-08-30T23:14:14.3Z,2018-08-23T01:15:12Z,"{'latitude': -36.0428871, 'longitude': 146.935...","{'canDisplayPrice': False, 'displayPrice': '$2...",NaN,NaN
152900,['townhouse'],rent,"{'stateAbbreviation': 'nsw', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,2.0,2.0,2018-09-26T03:12:36.243Z,2018-08-24T08:16:04Z,"{'latitude': -36.0384167, 'longitude': 146.938...","{'canDisplayPrice': False, 'displayPrice': '$2...",NaN,NaN
152901,['apartmentUnitFlat'],rent,"{'stateAbbreviation': 'nsw', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,2.0,1.0,2018-09-06T06:13:17.627Z,2018-08-29T07:14:29Z,"{'latitude': -36.0349289, 'longitude': 146.930...","{'canDisplayPrice': False, 'displayPrice': '$2...",NaN,NaN
152902,['townhouse'],rent,"{'stateAbbreviation': 'nsw', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,3.0,1.0,2020-05-22T01:12:51.707Z,2020-05-12T05:23:21Z,"{'latitude': -36.0454958, 'longitude': 146.913...","{'canDisplayPrice': False, 'displayPrice': '$2...",NaN,NaN


In [84]:
house_df.to_csv('../data/raw/house_df_new.csv')

In [422]:
house_df_2.to_csv('../data/raw/house_df_2.csv')

In [85]:
import re
suburb_df = []
for i in range(len(house_df['addressParts'])):
    s = re.findall(r'\'suburb\': +\'+[A-Za-z\' \']+',house_df['addressParts'][i])
    if(len(s) == 0):
        suburb_df.append(' ')
    else:
        s = s[0].split(':')[1][2:-1].lower()
        suburb_df.append(s)

In [424]:
import re
suburb_df_2 = []
for i in range(len(house_df_2['addressParts'])):
    s = re.findall(r'\'suburb\': +\'+[A-Za-z\' \']+',house_df_2['addressParts'][i])
    if(len(s) == 0):
        suburb_df_2.append(' ')
    else:
        s = s[0].split(':')[1][2:-1].lower()
        suburb_df_2.append(s)

In [86]:
postcode_df = []
for i in range(len(house_df['addressParts'])):
    postcode = re.findall(r'postcode\': \'[0-9]+',house_df['addressParts'][i])
    if(len(postcode) == 0):
        postcode_df.append(0)
    else:
        postcode = postcode[0].split(':')[1][2:]
        postcode_df.append(int(postcode))

In [425]:
postcode_df_2 = []
for i in range(len(house_df_2['addressParts'])):
    postcode = re.findall(r'postcode\': \'[0-9]+',house_df_2['addressParts'][i])
    if(len(postcode) == 0):
        postcode_df_2.append(0)
    else:
        postcode = postcode[0].split(':')[1][2:]
        postcode_df_2.append(int(postcode))

In [87]:
suburb_df = pd.DataFrame(suburb_df)
postcode_df = pd.DataFrame(postcode_df)
house_df['postcode'] = postcode_df
house_df['surburb'] = suburb_df

In [ ]:
suburb_df_2 = pd.DataFrame(suburb_df_2)
postcode_df_2 = pd.DataFrame(postcode_df_2)
house_df_2['postcode'] = postcode_df_2
house_df_2['surburb'] = suburb_df_2

In [88]:
year_df = []
month_df = []
for i in range(len(house_df['addressParts'])):
    date = house_df['dateUpdated'][i]
    if(len(date) == 0):
        year_df.append(0)
        month_df.append(0)
    else:
        year = date[:4]
        month = date[5:7]
        year_df.append(int(year))
        month_df.append(int(month))


In [429]:
year_df_2 = []
month_df_2 = []
for i in range(len(house_df_2['addressParts'])):
    date = house_df_2['dateUpdated'][i]
    if(len(date) == 0):
        year_df_2.append(0)
        month_df_2.append(0)
    else:
        year = date[:4]
        month = date[5:7]
        year_df_2.append(int(year))
        month_df_2.append(int(month))

In [89]:
year_df = pd.DataFrame(year_df)
month_df = pd.DataFrame(month_df)
house_df['year'] = year_df
house_df['month'] = month_df

In [430]:
year_df_2 = pd.DataFrame(year_df_2)
month_df_2 = pd.DataFrame(month_df_2)
house_df_2['year'] = year_df_2
house_df_2['month'] = month_df_2

In [90]:
house_df = house_df[house_df['year']>2018]
house_df

,propertyTypes,objective,addressParts,advertiserIdentifiers,bathrooms,bedrooms,carspaces,dateUpdated,dateListed,geoLocation,priceDetails,landAreaSqm,dateAvailable,postcode,surburb,year,month
105,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,3.0,1.0,2019-02-12T04:53:24.147Z,2014-06-04T04:40:45Z,"{'latitude': -36.3567799, 'longitude': 146.688...","{'canDisplayPrice': False, 'displayPrice': '$3...",NaN,NaN,3747,beechworth,2019,2
110,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,3.0,0.0,2020-03-27T03:59:01.72Z,2020-02-11T23:09:18Z,"{'latitude': -36.3672451, 'longitude': 146.681...","{'price': 340.0, 'pricePrefix': '$340 per week...",NaN,2020-03-20,3747,beechworth,2020,3
111,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,4.0,1.0,2019-11-24T23:57:24.163Z,2019-09-16T00:14:18Z,"{'latitude': -36.3571552, 'longitude': 146.687...","{'canDisplayPrice': False, 'displayPrice': '$3...",NaN,2019-10-18,3747,beechworth,2019,11
112,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,3.0,1.0,2020-05-05T05:32:13.28Z,2020-04-14T04:46:54Z,"{'latitude': -36.3545598, 'longitude': 146.689...","{'canDisplayPrice': False, 'displayPrice': '$3...",NaN,2020-05-08,3747,beechworth,2020,5
114,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,2.0,0.0,2019-11-25T04:02:27.697Z,2019-11-13T00:16:40Z,"{'latitude': -36.3560633, 'longitude': 146.689...","{'canDisplayPrice': False, 'displayPrice': '$2...",NaN,2019-12-19,3747,beechworth,2019,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59058,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,4.0,4.0,2022-09-05T23:17:47.523Z,2022-08-05T03:33:29Z,"{'latitude': -37.2844387, 'longitude': 144.728...","{'canDisplayPrice': False, 'displayPrice': 'UN...",NaN,2022-08-11,3435,lancefield,2022,9
59059,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,4.0,2.0,2022-09-27T23:34:22.997Z,2022-08-21T23:50:38Z,"{'latitude': -37.2844387, 'longitude': 144.728...","{'canDisplayPrice': False, 'displayPrice': 'UN...",567.91,2022-08-22,3435,lancefield,2022,9
59060,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",1.0,2.0,0.0,2022-09-27T04:46:42.247Z,2022-09-25T23:29:15Z,"{'latitude': -37.3460627, 'longitude': 144.741...","{'canDisplayPrice': False, 'displayPrice': '$4...",NaN,2022-10-28,3434,romsey,2022,9
59061,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,4.0,4.0,2022-10-03T23:30:24.74Z,2022-09-26T03:48:38Z,"{'latitude': -37.3626928, 'longitude': 144.747...","{'canDisplayPrice': False, 'displayPrice': 'UN...",813.00,2022-10-03,3434,romsey,2022,10


In [431]:
house_df_2 = house_df_2[house_df_2['year']>2018]
house_df_2 = house_df_2.reset_index()
house_df_2['year'].value_counts()

2021    9894
2022    9267
2019    8940
2020    7892
Name: year, dtype: int64

In [91]:
house_df = house_df.reset_index()
house_df['year'].value_counts()

2019    2383
2020    2215
2021    2112
2022    1893
Name: year, dtype: int64

In [92]:
def get_bond(text):
    text_dict = eval(text)
    return text_dict.get('bond')
def get_price(text):
    text_dict = eval(text)
    if text_dict.get('price'):
        return text_dict.get('price')
    else:
        return text_dict.get('displayPrice')

def to_lower(text):
    if isinstance(text,float):
        return text
    text = text.lower()
    return text

In [93]:
house_df['price'] = house_df['priceDetails'].apply(get_price)
house_df['bond'] = house_df['priceDetails'].apply(get_bond)
house_df['price'] = house_df['price'].apply(to_lower)

In [432]:
house_df_2['price'] = house_df_2['priceDetails'].apply(get_price)
house_df_2['bond'] = house_df_2['priceDetails'].apply(get_bond)
house_df_2['price'] = house_df_2['price'].apply(to_lower)

In [96]:
def get_weekly_price(text):
    if isinstance(text,float):
        return text
    money = re.findall(r'\$\d*[,.]?\d+[,.]?\d+',text)
    if len(money) == 0:
        return None
    elif len(money) == 1:
        money = money[0].replace(',','')[1:]
        if money.count('.') == 2:
            return None
        else:
            return float(money)
    else:
        if('per week' in text):
            num_text = re.findall(r'\$\d*[,.]?\d+[,.]?\d+ per week',text)
            if (len(num_text)== 1):
                return float(num_text[0].split(' ')[0][1:].replace(',',''))
            else:
                total = 0
                for i in num_text:
                    i = i.split(' ')[0][1:].replace(',','')
                    total += float(i)
                return total/len(num_text)
        elif('pw' in text):
            text1 = re.findall(r'\$\d*[,.]?\d+[,.]?\d+ pw',text)
            if(len(text1) == 1):
                return float(text1[0].split(' ')[0][1:].replace(',',''))
            elif(len(text1) > 1):
                total = 0
                for i in text1:
                    i = i.split(' ')[0][1:].replace(',','')
                    total += float(i)
                return total/len(text1)
            else:
                text2 = re.findall(r'\$\d*[,.]?\d+[,.]?\d+pw',text)
                if(len(text2)==1):
                    if(text2[0][0] != '$'):
                        return float(text2[0][:-2].replace(',',''))
                    else:
                        return float(text2[0][1:-2].replace(',',''))
                elif(len(text2) > 1):
                    total = 0
                    for i in text2:
                        i = i.split(' ')[0][1:-2].replace(',','')
                        total += float(i)
                    return total/len(text2)
        elif('p/w' in text):
            text3 = re.findall(r'\$\d*[,.]?\d+[,.]?\d+ p/w',text)
            if(len(text3)==1):
                text3 = text3[0].split(' ')[0][1:].replace(',','')
                return float(text3)
            elif(len(text3)>1):
                total = 0
                for i in text3:
                    i = i.split(' ')[0][1:-2].replace(',','')
                    total += float(i)
                return total/len(text3)
            else:
                text4 = re.findall(r'\$\d*[,.]?\d+[,.]?\d+p/w',text)
                if (len(text4)==1):
                    return float(text4[0][1:-3].replace(',',''))
                elif (len(text4) > 1):
                    total = 0
                    for i in text4:
                        i = i[1:-3].replace(',','')
                        total += float(i)
                    return total/len(text4)      

In [97]:
house_df['rent'] = house_df['price'].apply(get_weekly_price)
house_df.to_csv("../data/curated/house_df_new.csv")

In [98]:
house_df

,index,propertyTypes,objective,addressParts,advertiserIdentifiers,bathrooms,bedrooms,carspaces,dateUpdated,dateListed,...,priceDetails,landAreaSqm,dateAvailable,postcode,surburb,year,month,price,bond,rent
0,105,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,3.0,1.0,2019-02-12T04:53:24.147Z,2014-06-04T04:40:45Z,...,"{'canDisplayPrice': False, 'displayPrice': '$3...",NaN,NaN,3747,beechworth,2019,2,$320 per week,NaN,320.0
1,110,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,3.0,0.0,2020-03-27T03:59:01.72Z,2020-02-11T23:09:18Z,...,"{'price': 340.0, 'pricePrefix': '$340 per week...",NaN,2020-03-20,3747,beechworth,2020,3,340.0,1473.0,340.0
2,111,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,4.0,1.0,2019-11-24T23:57:24.163Z,2019-09-16T00:14:18Z,...,"{'canDisplayPrice': False, 'displayPrice': '$3...",NaN,2019-10-18,3747,beechworth,2019,11,$360 per week,NaN,360.0
3,112,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,3.0,1.0,2020-05-05T05:32:13.28Z,2020-04-14T04:46:54Z,...,"{'canDisplayPrice': False, 'displayPrice': '$3...",NaN,2020-05-08,3747,beechworth,2020,5,$350 per week,NaN,350.0
4,114,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,2.0,0.0,2019-11-25T04:02:27.697Z,2019-11-13T00:16:40Z,...,"{'canDisplayPrice': False, 'displayPrice': '$2...",NaN,2019-12-19,3747,beechworth,2019,11,$285 per week,NaN,285.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8598,59058,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,4.0,4.0,2022-09-05T23:17:47.523Z,2022-08-05T03:33:29Z,...,"{'canDisplayPrice': False, 'displayPrice': 'UN...",NaN,2022-08-11,3435,lancefield,2022,9,under application,NaN,NaN
8599,59059,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,4.0,2.0,2022-09-27T23:34:22.997Z,2022-08-21T23:50:38Z,...,"{'canDisplayPrice': False, 'displayPrice': 'UN...",567.91,2022-08-22,3435,lancefield,2022,9,under application,NaN,NaN
8600,59060,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",1.0,2.0,0.0,2022-09-27T04:46:42.247Z,2022-09-25T23:29:15Z,...,"{'canDisplayPrice': False, 'displayPrice': '$4...",NaN,2022-10-28,3434,romsey,2022,9,$440 per week,NaN,440.0
8601,59061,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,4.0,4.0,2022-10-03T23:30:24.74Z,2022-09-26T03:48:38Z,...,"{'canDisplayPrice': False, 'displayPrice': 'UN...",813.00,2022-10-03,3434,romsey,2022,10,under application,NaN,NaN


In [563]:
house_df_combined = pd.concat([house_df,house_df_2]).reset_index().drop(['level_0'],axis = 1)

In [564]:
house_df_combined 

,index,propertyTypes,objective,addressParts,advertiserIdentifiers,bathrooms,bedrooms,carspaces,dateUpdated,dateListed,...,priceDetails,landAreaSqm,dateAvailable,postcode,surburb,year,month,price,bond,rent
0,4256,['apartmentUnitFlat'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",1.0,1.0,0.0,2019-01-09T00:23:18.123Z,2016-02-16T04:33:49Z,...,"{'canDisplayPrice': False, 'displayPrice': '$3...",NaN,NaN,3205,south melbourne,2019,1,$360.00 per week,NaN,360.0
1,5429,['apartmentUnitFlat'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,4.0,4.0,2019-01-03T22:51:13.063Z,2018-10-02T00:00:38Z,...,"{'canDisplayPrice': False, 'displayPrice': '$1...",NaN,NaN,3207,port melbourne,2019,1,$1800.00 per week,NaN,1800.0
2,5437,['apartmentUnitFlat'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,2.0,1.0,2019-01-10T17:10:35.493Z,2018-10-05T04:57:13Z,...,"{'canDisplayPrice': False, 'displayPrice': '$5...",NaN,NaN,3207,port melbourne,2019,1,$550.00 per week,NaN,550.0
3,5439,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,3.0,0.0,2019-01-22T00:20:53.707Z,2018-10-05T07:07:14Z,...,"{'canDisplayPrice': False, 'displayPrice': '$9...",NaN,NaN,3206,albert park,2019,1,$975.00 per week,NaN,975.0
4,5447,['apartmentUnitFlat'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",1.0,2.0,1.0,2019-01-17T17:10:07.873Z,2018-10-17T01:02:26Z,...,"{'canDisplayPrice': False, 'displayPrice': '$5...",NaN,NaN,3207,port melbourne,2019,1,$500.00 per week,NaN,500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116286,152888,['house'],rent,"{'stateAbbreviation': 'nsw', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,3.0,1.0,2020-06-03T06:12:52.787Z,2020-04-28T05:13:10Z,...,"{'canDisplayPrice': False, 'displayPrice': '$2...",NaN,NaN,2641,lavington,2020,6,$210 per week,NaN,210.0
116287,152889,['apartmentUnitFlat'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,2.0,1.0,2019-08-08T03:32:08.4Z,2019-07-26T04:13:41Z,...,"{'canDisplayPrice': False, 'displayPrice': '$1...",NaN,NaN,3690,wodonga,2019,8,$190 per week,NaN,190.0
116288,152894,['house'],rent,"{'stateAbbreviation': 'nsw', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",2.0,4.0,2.0,2019-09-10T00:13:03.153Z,2019-08-08T11:13:16Z,...,"{'canDisplayPrice': False, 'displayPrice': '$3...",NaN,NaN,2640,west albury,2019,9,$315 per week,NaN,315.0
116289,152895,['house'],rent,"{'stateAbbreviation': 'nsw', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,3.0,2.0,2019-10-23T07:15:16Z,2019-10-03T05:12:50Z,...,"{'canDisplayPrice': False, 'displayPrice': '$3...",NaN,NaN,2641,lavington,2019,10,$300 per week,NaN,300.0


In [99]:
import ast
state = []
for i in range(len(house_df)): 
    s = ast.literal_eval(house_df['addressParts'][i])['stateAbbreviation']
    state.append(s)

In [100]:
house_df['state'] = state

In [101]:
house_df = house_df[house_df['state'] == 'vic'].drop('index', axis = 1)
house_df

,propertyTypes,objective,addressParts,advertiserIdentifiers,bathrooms,bedrooms,carspaces,dateUpdated,dateListed,geoLocation,...,landAreaSqm,dateAvailable,postcode,surburb,year,month,price,bond,rent,state
0,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,3.0,1.0,2019-02-12T04:53:24.147Z,2014-06-04T04:40:45Z,"{'latitude': -36.3567799, 'longitude': 146.688...",...,NaN,NaN,3747,beechworth,2019,2,$320 per week,NaN,320.0,vic
1,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,3.0,0.0,2020-03-27T03:59:01.72Z,2020-02-11T23:09:18Z,"{'latitude': -36.3672451, 'longitude': 146.681...",...,NaN,2020-03-20,3747,beechworth,2020,3,340.0,1473.0,340.0,vic
2,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,4.0,1.0,2019-11-24T23:57:24.163Z,2019-09-16T00:14:18Z,"{'latitude': -36.3571552, 'longitude': 146.687...",...,NaN,2019-10-18,3747,beechworth,2019,11,$360 per week,NaN,360.0,vic
3,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,3.0,1.0,2020-05-05T05:32:13.28Z,2020-04-14T04:46:54Z,"{'latitude': -36.3545598, 'longitude': 146.689...",...,NaN,2020-05-08,3747,beechworth,2020,5,$350 per week,NaN,350.0,vic
4,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,2.0,0.0,2019-11-25T04:02:27.697Z,2019-11-13T00:16:40Z,"{'latitude': -36.3560633, 'longitude': 146.689...",...,NaN,2019-12-19,3747,beechworth,2019,11,$285 per week,NaN,285.0,vic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8598,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,4.0,4.0,2022-09-05T23:17:47.523Z,2022-08-05T03:33:29Z,"{'latitude': -37.2844387, 'longitude': 144.728...",...,NaN,2022-08-11,3435,lancefield,2022,9,under application,NaN,NaN,vic
8599,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,4.0,2.0,2022-09-27T23:34:22.997Z,2022-08-21T23:50:38Z,"{'latitude': -37.2844387, 'longitude': 144.728...",...,567.91,2022-08-22,3435,lancefield,2022,9,under application,NaN,NaN,vic
8600,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",1.0,2.0,0.0,2022-09-27T04:46:42.247Z,2022-09-25T23:29:15Z,"{'latitude': -37.3460627, 'longitude': 144.741...",...,NaN,2022-10-28,3434,romsey,2022,9,$440 per week,NaN,440.0,vic
8601,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,4.0,4.0,2022-10-03T23:30:24.74Z,2022-09-26T03:48:38Z,"{'latitude': -37.3626928, 'longitude': 144.747...",...,813.00,2022-10-03,3434,romsey,2022,10,under application,NaN,NaN,vic


In [103]:
house_df.to_csv('../data/house_combined_new.csv')